# Scrapping

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re, time,csv
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import json
import os


In [2]:
#Function to extract data from  webpage
def get_links_from_page(
    writer:csv.writer
            ):
    
    
    #get all story divs in the current page
    results=driver.find_elements(by=By.CSS_SELECTOR,value="li.o-listicle__item")
    #print('result', results)
    #hide_ads()
    for result in results: 
        
        Date, Platform, Text, Person, Truthfulness_Category='NA','NA','NA','NA','NA'
        
        try: # try to get the date AND platform of the statement
            
            Date_platform_box=result.find_element(by=By.CSS_SELECTOR,value='div.m-statement__desc')
            Date = Date_platform_box.text.split(' ')[2:5] #need to check
            Date = ' '.join(Date)
            Platform=Date_platform_box.text.split(' ')[6:]
            Platform = ' '.join(Platform)
            print('Date', Date)
            print('Platform', Platform)
        except NoSuchElementException as e: #  could not be found
            print('could not extract Date and Platform')
            
        try: #try to get the text of the statement
            Text_box=result.find_element(by=By.CSS_SELECTOR,value='div.m-statement__quote')
            Text=Text_box.text
            print('Text:', Text)
            
        except NoSuchElementException as e: # could not be found
            print('could not extract Text of the statement')
    
            
        try: # try to get the person who made the statement

            Person_box=result.find_element(by=By.CSS_SELECTOR,value='a.m-statement__name')
            Person_name=Person_box.text
            print('Person:', Person_name)

        except NoSuchElementException as e:
            print('could not extract the person who made the statement')
            
#-----------------------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------------------
####BELOW IS EXTRACTING THE TRUTHFULNESS CATEGORY
        try:  #try to get the statement truthfulness category - FALSE
            Truthfulness_box1 = result.find_element(by=By.CSS_SELECTOR, value='img[alt="false"]')
            
            Truthfulness_Category1 = 'False'
            print('Truthfulness_Category', Truthfulness_Category1)
            Truthfulness_Category = Truthfulness_Category1
    
        except:
            pass
            
        try:  #try to get the statement truthfulness category - PANTS ON FIRE
            Truthfulness_box2 = result.find_element(by=By.CSS_SELECTOR, value='img[alt="pants-fire"]')
            
            Truthfulness_Category2 = 'Pants on Fire'
            print('Truthfulness_Category', Truthfulness_Category2)
            Truthfulness_Category = Truthfulness_Category2
        except:
            pass
            
        try:  #try to get the statement truthfulness category - MOSTLY FALSE
            Truthfulness_box3 = result.find_element(by=By.CSS_SELECTOR, value='img[alt="barely-true"]')  #notice this is barely-true in the webpage html and the picture is shown as mostly false
            
            Truthfulness_Category3 = 'Mostly False'
            print('Truthfulness_Category', Truthfulness_Category3)
            Truthfulness_Category = Truthfulness_Category3
        except:
            pass

        try:  #try to get the statement truthfulness category - MOSTLY TRUE
            Truthfulness_box4 = result.find_element(by=By.CSS_SELECTOR, value='img[alt="mostly-true"]')
            
            Truthfulness_Category4 = 'Mostly True'
            print('Truthfulness_Category', Truthfulness_Category4)
            Truthfulness_Category = Truthfulness_Category4
        except:
            pass

        try:  #try to get the statement truthfulness category -TRUE
            Truthfulness_box5 = result.find_element(by=By.CSS_SELECTOR, value='img[alt="true"]')
            
            Truthfulness_Category5 = 'True'
            print('Truthfulness_Category', Truthfulness_Category5)
            Truthfulness_Category = Truthfulness_Category5
        except:
            pass

        writer.writerow([Date,Platform,Text, Person_name, Truthfulness_Category])


In [3]:
#function to extract data from every page
def scrape_query(url):
    driver.get(url)
    page_count = 1
    
    fw=open('data.csv','w',encoding='utf-8')
    writer=csv.writer(fw,lineterminator='\n')
    writer.writerow(['Date', 'Platform', 'Text', 'Person', 'Truthfulness_Category'])
    
    get_links_from_page(writer)
    while True:
        for page_count in range(1,300):
            try:
                
                driver.refresh()
                time.sleep(5)
                
                #Remove all Google Ads pop ups
                all_iframes = driver.find_elements(by = By.TAG_NAME, value="iframe")
                if len(all_iframes) > 0:
                    print("Ad Found\n")
                    driver.execute_script("""
                        var elems = document.getElementsByTagName("iframe"); 
                        for(var i = 0, max = elems.length; i < max; i++)
                             {
                                 elems[i].hidden=true;
                             }
                                          """)
                    print('Total Ads: ' + str(len(all_iframes)))
                else:
                    print('No frames found')

                chromeOpt = webdriver.ChromeOptions()
                appState = {
                   "recentDestinations": [
                        {
                            "id": "Save as PDF",
                            "origin": "local",
                            "account": ""
                        }
                    ],
                    "selectedDestinationId": "Save as PDF",
                    "version": 2,
                    "isHeaderFooterEnabled": False
                }
                prefs = {
                    'printing.print_preview_sticky_settings.appState': json.dumps(appState)}
                chromeOpt.add_experimental_option('prefs', prefs)
                chromeOpt.add_argument('--kiosk-printing')

            
                
                results=driver.find_elements(by=By.CSS_SELECTOR,value='div[class = "o-platform__content"]')
                for result in results:
                    
                
                    name = result.find_element(By.LINK_TEXT, 'Next')
                    
                    
                    
                    name.click()
                    get_links_from_page(writer)
                    print("page",page_count)
                    page_count+=1
            #         time.sleep(5)
                    print(name)
                    
                    
            except:
                print("Ad popped up")
        break

In [4]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()
url = "https://www.politifact.com/factchecks/list/?page=1&category=truth-o-meter"
scrape_query(url)

Date September 15, 2008
Platform a news release.:
Text: "Barack Obama has the longest track record of any candidate in this election in support of the 'Bridge To Nowhere.' "
Person: John McCain
Truthfulness_Category Mostly False
Date September 22, 2008
Platform a TV ad.:
Text: John McCain would "reduce oversight of health insurance."
Person: Barack Obama
Date September 12, 2008
Platform blog postings on many Web sites.:
Text: Sarah Palin endorsed a Wasilla policy that charged sexual assault victims for their "rape kits."
Person: Bloggers
Date September 20, 2008:
Platform 
Text: "But if my opponent had his way, the millions of Floridians who rely on it would've had their Social Security tied up in the stock market this week."
Person: Barack Obama
Truthfulness_Category False
Date September 18, 2008
Platform a video ad.:
Text: Obama ad contends that John McCain endorses Rush Limbaugh's comments about immigration.
Person: Barack Obama
Truthfulness_Category Pants on Fire
Date September 17, 